Imports and data preparation

In [1]:
%matplotlib inline
import pandas as pd
import quandl , math ,datetime
import numpy as np
from sklearn import preprocessing , model_selection , svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

import matplotlib.pyplot as plt

from matplotlib import style

style.use('ggplot')


df = pd.read_csv("/home/balerion/Desktop/Ai/GOOGL.csv")

# print(df.head())

df = df[["Open" , "High" , "Low" , "Close" , "Volume"]]

# df["HL_PCT"] = (df["High"] - df["Low"]) / df["Low"] * 100

df["HC_PCT"] = (df["High"] - df["Close"]) / df["Close"] * 100.0

df["PCT_change"] = (df["Close"] - df["Open"]) / df["Open"] * 100.0

df = df[["Close" , "HC_PCT" , "PCT_change" , "Volume"]]

# print(df.head())


forecast_col = "Close"

df.fillna(-99999 , inplace=True) # to get rid of nill data

forecast_out = int(math.ceil(0.01*len(df))) # for every 40 odd days

print(forecast_out)

df["label"] = df[forecast_col].shift(-forecast_out)

# print(df.head())



45


Prprocessing and Scaling

In [2]:
x = np.array(df.drop(columns=["label"]))# x = np.array(df.drop(columns = ["label"])) -> contains features
x = preprocessing.scale(x)
x = x[:-forecast_out]

x_lately = x[-forecast_out:]

# standardization of data x' = x - mean / standard deviation
# Standardized data refers to data that has been transformed to have a mean of 0 and a standard deviation of 1
# x = x[:-forecast_out+1]

df.dropna(inplace=True)
y = np.array(df["label"])
y = np.array(df["label"])

print(len(x) , len(y))



4386 4386


Why Scaling is important ?


In [3]:
# import numpy as np
# import pandas as pd
# from sklearn.preprocessing import StandardScaler

# # Sample DataFrame
# data = {
#     'feature1': [1, 2, 3, 4, 5],
#     'feature2': [10, 20, 30, 40, 50],
#     'feature3': [100, 200, 300, 400, 500]
# }
# df = pd.DataFrame(data)

# # Extract features
# x = df.values

# # Standardize the features
# scaler = StandardScaler()
# x_standardized = scaler.fit_transform(x)

# print("Original data:\n", x)
# print("Standardized data:\n", x_standardized)


#  Q. Why Standardization is Important ?

#  -> Improved Algorithm Performance:
# Many machine learning algorithms, such as those using gradient descent (e.g., linear regression, logistic regression) or distance metrics (e.g., k-nearest neighbors, support vector machines), perform better or converge faster with standardized data.

#  -> Fair Feature Contribution: 
# Standardization ensures that each feature contributes equally to the model. Without standardization, features with larger ranges can dominate the learning process, leading to biased results.

#  -> Stable and Faster Convergence: 
# For optimization algorithms, such as gradient descent, standardized data ensures stable and faster convergence by providing a smoother cost function landscape.

Model training and testing

In [4]:
x_train , x_test , y_train , y_test = model_selection.train_test_split(x , y , test_size=0.2)
classifier =  LinearRegression(n_jobs=10)   #svm.SVR(kernel = "poly")

classifier.fit(x_train , y_train)  # fit("used for training the model")

accuracy = classifier.score(x_test , y_test) # score("used for testing accuracy")

forecast_set = classifier.predict(x_lately)

print(forecast_set , accuracy , forecast_out)

# print(accuracy)


[3076.76667495 3064.48927242 3067.07412836 3108.837462   3088.982293
 3036.5597913  3020.62575554 3027.16007482 2948.33613131 3019.87405106
 2947.78548598 2934.80608192 2968.14368041 2945.67239743 2966.64801586
 3053.87957083 3069.934959   3062.91193457 3066.2157613  3023.23202745
 2980.90997934 3040.44670189 2997.53453969 2940.75301046 2936.36572337
 2975.43488267 3040.19102454 3048.26479651 3065.60166211 3039.90203507
 3039.44628214 3030.32495436 3003.9618907  3004.96681687 2995.94956629
 2865.87519271 2863.71596991 2839.82800475 2880.83047498 2899.49885615
 2933.85795778 2881.02333482 2899.89799678 2819.82353426 2806.80432766] 0.9818354506150209 45


What is SVR ?

In [5]:
# Key Concepts of SVR

# Margin of Tolerance : SVR introduces a margin of tolerance, within which errors are ignored. The goal is to find a function that deviates from the actual data points by a value no greater than tolerance.

# Support Vectors: Data points that lie within the margin boundaries and influence the position and orientation of the regression line.

# Kernel Trick: SVR can use kernel functions (like linear, polynomial, or RBF) to handle non-linear relationships by mapping input features into higher-dimensional spaces.